In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Loading data

In [4]:
df_pro_contacts_adults = pd.read_csv("pro_contacts_adults.csv")
df_pro_contacts_children = pd.read_csv("pro_contacts_children.csv")
df_households = pd.read_csv("households.csv")